In [0]:
import pandas as pd
import re
from gensim import corpora
from gensim import similarities
from gensim.matutils import jaccard
from gensim.utils import simple_preprocess
from gensim import models
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pd.set_option('display.max_colwidth', 500)
# tweets = pd.read_csv('../Data/tweets.csv')
# tweets_cid = pd.read_csv('../Data/tweetsCID.csv')
# knights_url = pd.read_csv('../Data/knight_data_urls.csv')

tweets = pd.read_csv('/content/drive/My Drive/Documents/0_Columbia/9_Capstone/Data/tweets.csv')
tweets_cid = pd.read_csv('/content/drive/My Drive/Documents/0_Columbia/9_Capstone/Data/tweetsCID.csv')
knights_url = pd.read_csv('/content/drive/My Drive/Documents/0_Columbia/9_Capstone/Data/knight_data_urls.csv')

In [4]:
# tweets.info()
tweets_cid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201873 entries, 0 to 201872
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   tweet_id     201873 non-null  int64 
 1   tweet_text   201873 non-null  object
 2   user_id      201873 non-null  int64 
 3   tweet_time   201873 non-null  object
 4   no_favs      201873 non-null  int64 
 5   no_retweets  201873 non-null  int64 
 6   urls         201815 non-null  object
 7   content_id   201873 non-null  object
dtypes: int64(4), object(4)
memory usage: 12.3+ MB


In [5]:
tweets_cid.groupby(['content_id'])['tweet_id'].count().sort_values(ascending=False)

content_id
3045    4135
1775    3736
2360    3221
2661    2350
2408    2312
        ... 
2001       1
466        1
1840       1
2484       1
2483       1
Name: tweet_id, Length: 1474, dtype: int64

## Data Prep

#### Drop duplicates based on same Tweet_Id

In [0]:
tweets_cid = tweets_cid.drop_duplicates(subset='tweet_id', keep="first")

In [7]:
print(f'Number of rows now: {tweets_cid.shape[0]:,}')

Number of rows now: 198,487


#### Substituting the URLs with \<URL> token using regex
 * Adding a field with a URL token instead of the URL and another field that has the list of URLs extracted. 
 * Adding another column with preprocessed tweet text to remove punctuations

In [9]:
tweets_cid['tweet_text_url_token'] = \
tweets_cid['tweet_text'].apply(lambda x:re.sub(r'(https?://[www\.]?\w*\.[\w/$]*)', '<URL>', x))

tweets_cid['tweet_urls'] = \
tweets_cid['tweet_text'].apply(lambda x:re.findall(r'(https?://[www\.]?\w*\.[\w/$]*)', x))

tweets_cid['tweet_text_url_token_preprocessed'] = tweets_cid['tweet_text_url_token'].apply(lambda x:simple_preprocess(x))

tweets_cid[:5]

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id,tweet_text_url_token,tweet_urls,tweet_text_url_token_preprocessed
0,851485433003421701,#Trump’s border wall would be an ecological and financial disaster for the U.S. https://t.co/lGSc8X53Vg,613521891,2017-04-10T17:22:01.000Z,0,0,https://www.vox.com/energy-and-environment/2017/4/10/14471304/trump-border-wall-animals,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,[https://t.co/lGSc8X53Vg],"[trump, border, wall, would, be, an, ecological, and, financial, disaster, for, the, url]"
1,851514117374398464,#Trump’s border wall would be an ecological and financial disaster for the U.S. https://t.co/v1TW467479,222481411,2017-04-10T19:16:00.000Z,0,0,https://www.vox.com/energy-and-environment/2017/4/10/14471304/trump-border-wall-animals,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,[https://t.co/v1TW467479],"[trump, border, wall, would, be, an, ecological, and, financial, disaster, for, the, url]"
2,855009400670617600,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking https://t.co/ClfsLDuDCO,2347049341,2017-04-20T10:45:01.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4/19/15326286/trump-wall-opioid-epidemic,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,[https://t.co/ClfsLDuDCO],"[the, research, and, experts, are, clear, tougher, border, security, measures, do, not, actually, stop, drug, trafficking, url]"
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4/19/15326286/trump-wall-opioid-epidemic,2644,Hmmmmmm <URL>,[https://t.co/y3fNx7KuJT],"[hmmmmmm, url]"
4,855011873183342592,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) https://t.co/cMeFEnipLW,17889654,2017-04-20T10:54:50.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4/19/15326286/trump-wall-opioid-epidemic,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,[https://t.co/cMeFEnipLW],"[psst, it, was, never, about, drugs, it, always, been, about, brown, people, they, holler, drugs, to, distract, from, their, horrible, racism, url]"


#### Drop duplicates based on same Tweet_Text after replacing the URL with the URL token

In [10]:
tweets_cid = tweets_cid.drop_duplicates(subset=['tweet_text_url_token','content_id'])
print(f'Number of rows now: {tweets_cid.shape[0]:,}')

Number of rows now: 132,159


## Identify Duplicate Tweets (by Content ID) and remove them

### Group Tweets By Content ID

In [12]:
content_ids = pd.DataFrame(tweets_cid.groupby(['content_id'])['tweet_id'].count())
content_ids.index.name = 'content_id'
content_ids.reset_index(inplace=True)
content_ids.rename(columns = {'tweet_id':'tweet_counts'}, inplace = True) 
content_ids = content_ids[content_ids['content_id'] !='blank']
content_ids[:10]

,content_id,tweet_counts
0,101,29
1,1019,13
2,102,15
3,103,15
4,1060,2
5,1081,45
6,1083,16
7,1090,1
8,1106,441
9,1113,128


In [13]:
tweets_cid[tweets_cid['content_id']== '101'][:5]

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id,tweet_text_url_token,tweet_urls,tweet_text_url_token_preprocessed
3062,902182893308575744,CNBC lets Bush's disgraced FEMA director exploit hurricane to pitch privatizing National Flood Insurance Program https://t.co/S8GeC0qwiO https://t.co/5U8Q2GsqPT,13493302,2017-08-28T14:55:38.000Z,0,0,https://www.mediamatters.org/video/2017/08/28/cnbc-lets-bushs-disgraced-fema-director-exploit-hurricane-pitch-privatizing-national-flood-insurance/217783,101,CNBC lets Bush's disgraced FEMA director exploit hurricane to pitch privatizing National Flood Insurance Program <URL> <URL>,"[https://t.co/S8GeC0qwiO, https://t.co/5U8Q2GsqPT]","[cnbc, lets, bush, disgraced, fema, director, exploit, hurricane, to, pitch, privatizing, national, flood, insurance, program, url, url]"
3063,902182884932558849,They did a heckuva job turning to him for insights... 🙄 https://t.co/4VItO0vjDC,1704646470,2017-08-28T14:55:36.000Z,0,0,https://www.mediamatters.org/video/2017/08/28/cnbc-lets-bushs-disgraced-fema-director-exploit-hurricane-pitch-privatizing-national-flood-insurance/217783,101,They did a heckuva job turning to him for insights... 🙄 <URL>,[https://t.co/4VItO0vjDC],"[they, did, heckuva, job, turning, to, him, for, insights, url]"
3064,902183119519965185,Idk maybe we shouldn't let the guy who let Katrina happen have a say in this one. https://t.co/YEz571bUaE,400160847,2017-08-28T14:56:32.000Z,0,0,https://www.mediamatters.org/video/2017/08/28/cnbc-lets-bushs-disgraced-fema-director-exploit-hurricane-pitch-privatizing-national-flood-insurance/217783,101,Idk maybe we shouldn't let the guy who let Katrina happen have a say in this one. <URL>,[https://t.co/YEz571bUaE],"[idk, maybe, we, shouldn, let, the, guy, who, let, katrina, happen, have, say, in, this, one, url]"
3065,902183336633913345,"Heckuva job, there, CNBC https://t.co/xJQ86OMTEk",874139012,2017-08-28T14:57:23.000Z,0,0,https://www.mediamatters.org/video/2017/08/28/cnbc-lets-bushs-disgraced-fema-director-exploit-hurricane-pitch-privatizing-national-flood-insurance/217783,101,"Heckuva job, there, CNBC <URL>",[https://t.co/xJQ86OMTEk],"[heckuva, job, there, cnbc, url]"
3066,902182264251002880,Somebody throw this man in the f***ing ocean https://t.co/SeTKAvpR4G,59788234,2017-08-28T14:53:08.000Z,0,0,https://www.mediamatters.org/video/2017/08/28/cnbc-lets-bushs-disgraced-fema-director-exploit-hurricane-pitch-privatizing-national-flood-insurance/217783,101,Somebody throw this man in the f***ing ocean <URL>,[https://t.co/SeTKAvpR4G],"[somebody, throw, this, man, in, the, ing, ocean, url]"


### Create Dictonary and Corpus using Gensim

In [0]:
# Punctuations Not Removed
def create_docs(source):
    return_dict = {}
    for idx,row in source.iterrows():
        return_dict[row['tweet_id']] = [word for word in row['tweet_text_url_token'].lower().split()]
        
    return return_dict

# Punctuations Removed
def create_docs_preprocessed(source):
    return_dict = {}
    for idx,row in source.iterrows():
        return_dict[row['tweet_id']] = [word for word in row['tweet_text_url_token_preprocessed']]
        
    return return_dict

def create_dictionary_corpus(source_dict):
    dictionary = corpora.Dictionary()
    corpus = []
    
    for key, doc in source_dict.items():
        texts = [doc]
        dictionary.add_documents(texts)
        corpus.append(dictionary.doc2bow(texts[0]))
        
    return dictionary, corpus

### Calculate Similarity Scores

In [0]:
def calculate_similarity_scores(score_type, content_ids):
    
    start_time = time.time()
    score_list = []

    for id in content_ids['content_id']:

        tweets_per_id = tweets_cid[tweets_cid['content_id']==id]
        tweets_per_id = tweets_per_id [['content_id', 'tweet_text_url_token', 'tweet_text_url_token_preprocessed', 'tweet_id']]

    #     tweets_dict = create_docs(tweets_per_id)  ## punctuations not removed
        tweets_dict = create_docs_preprocessed(tweets_per_id)  ## punctuations removed
        dictionary, corpus = create_dictionary_corpus(tweets_dict)
        
#         print(len(tweets_dict))
        # COSINE Similarity Score using bag of words
        if score_type == 'cosine_bagofwords':
            index_cosine = similarities.MatrixSimilarity(corpus)
            sims_cosine = index_cosine[corpus]
            score_list.append(sims_cosine)

        # COSINE Similarity Score using TF-IDF representation
        if score_type == 'cosine_tfidf':
            tfidf = models.TfidfModel(corpus)
            index_tfidf = similarities.MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))
            sims_tfidf = index_tfidf[tfidf[corpus]]
            score_list.append(sims_tfidf)

    print("---- %s seconds ----" % (time.time() - start_time ))
    return score_list

In [0]:
# jaccard(tweets_cid['tweet_text_url_token_preprocessed'].iloc[1], tweets_cid['tweet_text_url_token_preprocessed'].iloc[2])
# tweets_cid['tweet_text_url_token_preprocessed'].iloc[2]

### Create Clusters Of Similar Tweets (within Content_ID) based on similarity score threshold 

In [0]:
def create_clusters (score_list, content_ids, threshold):
    
    start_time = time.time()
    cluster_dict = {}

    for i in range(len(score_list)):

        score_array = score_list[i]
        content_id = list(content_ids['content_id'])[i]
        # content_id = content_ids['content_id'][i]

        # print ("Index = ", i)
        # print ("Content_Id = ", content_id)
        # print (score_array.shape)

        all_tweets = list(tweets_cid[tweets_cid['content_id']==content_id]['tweet_id'])
        all_tweets_text = list(tweets_cid[tweets_cid['content_id']==content_id]['tweet_text_url_token'])

        clustered_tweets = []
        unprocessed_tweets = []
        
        unprocessed_tweets = all_tweets
        length = len(unprocessed_tweets)
        
        # print(length)
        
        cnt = 0
        
        while length > 0:

        #     print ("cluster #", cnt)
        #     print (length)
        #     cnt = cnt+1

            dup_tweets = []

            tweet_a = unprocessed_tweets[0]
            tweet_a_idx = all_tweets.index(tweet_a)
            dup_tweets.append(tweet_a)

            if length > 1:
                for col in range(1, len(unprocessed_tweets)):

                    tweet_b = unprocessed_tweets[col]
                    tweet_b_idx = all_tweets.index(tweet_b)

                    if score_array[tweet_a_idx][tweet_b_idx] >= threshold:
                        dup_tweets.append(tweet_b)        
        #             break
            clustered_tweets.append(dup_tweets)

            unprocessed_tweets = list(set(unprocessed_tweets) - set(dup_tweets))
            length = len(unprocessed_tweets)

        cluster_dict[content_id] = clustered_tweets
    #     break
    print("---- %s seconds ----" % (time.time() - start_time ))
    return cluster_dict

### Remove Duplicates by pick one tweet from each cluster (within Content_Id)

In [0]:
def remove_dups(cluster_dict):
    
    start_time = time.time()
    no_dups_df = tweets_cid.iloc[0:0,:].copy()
    cnt = 0
    for i in cluster_dict :
        # print (cnt)
        cnt = cnt + 1
        
        content_id = i
        clusters = cluster_dict[i]
#         print("Content_Id = ", i)
        
        for j in range(len(clusters)):
#             print("Cluster # ", j)
            first_tweet_id  = clusters[j][0]
#             print(first_tweet_id)
            
            row_df = tweets_cid[(tweets_cid['tweet_id']==first_tweet_id) & (tweets_cid['content_id']==content_id)]
            no_dups_df = pd.concat([row_df, no_dups_df], ignore_index=True)

#         break
#     print(len(no_dups_df))
    print("---- %s seconds ----" % (time.time() - start_time ))
    return no_dups_df

In [0]:
def print_clusters(cluster_dict):
    
    for i in cluster_dict : 
        print("Content_Id = ", i)
        clusters = cluster_dict[i]
        for j in range(len(clusters)):
            print("Cluster # ", j)
            dup_cluster = clusters[j]
            for k in range(len(dup_cluster)):
                tweet_text = tweets_cid[tweets_cid['tweet_id']==dup_cluster[k]]['tweet_text_url_token']
                print("Tweet Id = ", dup_cluster[k])
                print("Tweet Text = ", tweet_text)
        break        
        

In [0]:
# ## SAMPLE TEST
# threshold = 0.60
# content_ids_tmp = content_ids[content_ids['content_id'] == '2168']
# # content_ids_tmp = content_ids [100:110]
# score_list_tmp = calculate_similarity_scores('cosine_bagofwords', content_ids_tmp)
# # print (score_list_tmp[0].shape)
# cosine_bow_cluster_dict = create_clusters(score_list_tmp, content_ids_tmp, threshold)

## COSINE Similarity using bag of words

In [0]:
cosine_bow_threshold = 0.60
cosine_bow_content_ids = content_ids
# cosine_bow_content_ids = content_ids[content_ids['tweet_counts'] > 2000]
# cosine_bow_content_ids = content_ids[content_ids['content_id'] == '2168']

print("Calculate Similarity Score")
cosine_bow_score_list   = calculate_similarity_scores('cosine_bagofwords', cosine_bow_content_ids)

print("Create Clusters")
cosine_bow_cluster_dict = create_clusters(cosine_bow_score_list, cosine_bow_content_ids, cosine_bow_threshold)

print("Remove Duplicates")
tweet_cid_no_dups_cosine_bow = remove_dups(cosine_bow_cluster_dict)

print("Original Records #", len(tweets_cid))
print("After Removing Dups #", len(tweet_cid_no_dups_cosine_bow))

tweet_cid_no_dups_cosine_bow.to_csv('/content/drive/My Drive/Documents/0_Columbia/9_Capstone/Data/tweetsCID_cosine_bow.csv', index = False)

Calculate Similarity Score


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


---- 67.15986394882202 seconds ----
Create Clusters
---- 214.47226285934448 seconds ----
Remove Duplicates


## COSINE Similarity using TF-IDF

In [0]:
cosine_tfidf_threshold = 0.30
cosine_tfidf_content_ids = content_ids
# cosine_tfidf_content_ids = content_ids[content_ids['tweet_counts'] > 2000]
# cosine_tfidf_content_ids = content_ids[content_ids['content_id'] == '2168']

print("Calculate Similarity Score")
cosine_tfidf_score_list   = calculate_similarity_scores('cosine_tfidf', cosine_tfidf_content_ids)

print("Create Clusters")
cosine_tfidf_cluster_dict = create_clusters(cosine_tfidf_score_list, cosine_tfidf_content_ids, cosine_tfidf_threshold)

print("Remove Duplicates")
tweet_cid_no_dups_cosine_tfidf = remove_dups(cosine_tfidf_cluster_dict)

print("Original Records #", len(tweets_cid))
print("After Removing Dups #", len(tweet_cid_no_dups_cosine_tfidf))

tweet_cid_no_dups_cosine_tfidf.to_csv('/content/drive/My Drive/Documents/0_Columbia/9_Capstone/Data/tweetsCID_cosine_tfidf.csv', index = False)